## Example overview

This notebook shows how to use Weave to build text extraction capabilities using LLMs.

It covers:
- [x] experimenting with different techniques for text extraction
- [x] using LLMs for text extraction
- [x] prompt experimentation
- [x] rigorous evaluation of text extraction models
- [ ] model serving and monitoring
- [ ] production feedback capture
- [ ] improving evaluation datasets from production data
- [ ] evolving the problem definition

All of the above is tracked and versioned using Weave, and presented in Weave's UI for analysis.

### Setup

Install the weave package from the weaveflow branch:

```
pip install git+https://github.com/wandb/weave@weaveflow
```

Run the prototype UI locally:

```
weave ui
```

In [ ]:
import glob
import os
import json
import typing
import weave

Everything will be tracked in the following W&B project, which will be auto-created if it doesn't exist.

In [ ]:
PROJECT = 'text-extract52'
weave.init(PROJECT)

This example shows extracting fields from "Articles of Incorporation" documents (these are legal documents that filed are when a company is formed). The example documents are generated by gpt-4.

In [ ]:
!ls example_data

An example document looks like this

In [ ]:
example_doc = open('example_data/Articles_of_Incorporation_Real_Example_3.txt').read()
print(example_doc)

The labels look like this. For now we want to extract the company name, and the initial number of stock shares. Only a subset of the example documents have labels.

In [ ]:
json.load(open(os.path.join("example_data", "labels.json")))

## Datasets

We create table with "example" and "label" columns, and then publish it with weave to start versioning it.

In [ ]:
# Read in our dataset
def read_dataset():
    dataset_rows = []
    raw_labels = json.load(open(os.path.join("example_data", "labels.json")))
    for example_id, label in raw_labels.items():
        example = open(os.path.join('example_data', example_id + '.txt')).read()
        dataset_rows.append(
            {"id": example_id, "example": example, "label": label})
    return dataset_rows

# Construct and publish to W&B
dataset = weave.Dataset(read_dataset())
dataset_ref = weave.publish(dataset, "eval_dataset")

In [ ]:
whatever_ref = weave.publish({'hi': 'adam'}, 'my-stuff')

In [ ]:
weave.ref('my-stuff').get()

Click the link printed above to view the published dataset in the UI.

### Editing data in the UI

There are a few missing labels! Double-click the table cells to edit the data in the UI, and fix the labels. Then press the commit version to commit the changes.

To get the fixed dataset back in Python, you can grab the latest version with the following command

In [ ]:
dataset_ref = weave.ref('eval_dataset')
dataset_ref.get().rows[2]['label']

## Tracking function calls with weave.op

Annotate a python function with weave.op to keep track of its code, log and log traces of its calls.

This is a simple baseline that uses regexes to try to extract the fields we want.

In [ ]:
import re

def predict_name(doc: str) -> typing.Any:
    match = re.search(r'name.*is ([^.]*)(\.|\n)', doc)
    return match.group(1) if match else None

def predict_shares(doc: str) -> typing.Any:
    match = re.search(r'[Ss]hares.*?([\d,]+)', doc)
    return match.group(1).replace(',', '') if match else None

@weave.op()
def predict(doc: str) -> typing.Any:
    return {
        'name': predict_name(doc),
        'shares': predict_shares(doc)
    }

Ops behave like normal functions. But their code is captured and versioned, and their calls are logged.

In [ ]:
predict(example_doc)

Click the link printed above to see all the calls of our op.

If you change the predict function's code by editing it and rerunning the jupyter cell where it's defined, the next time you call it you'll get a new version of the op. Try it!

In [ ]:
# Here we iterate through all the rows in the dataset, performing predictions
for row in dataset_ref.get().rows:
    print(predict(row['example']))

Go back to the UI to see all the calls we just made.

## Models

A "model" is simply a combination of data (which can be configuration, trained model weights, or anything else), and code that says how to execute the model.

Use the pattern below to construct a model.

The `@weave.type()` decorator makes classes that are automatically published and versioned as they are used.
- Like python's dataclasses feature, you must annotate attributes with python types.
- You can add methods that are weave ops.

Inherit from weave.Model to categorize this object as a Model in the UI.

In [ ]:
@weave.type()
class RegexModel(weave.Model):
    # Data: configuration for this model
    extract_name: bool
    extract_shares: bool

    # Code: how should this model execute on an example?
    @weave.op()
    def predict(self, doc: str) -> typing.Any:
        return {
            'name': predict_name(doc) if self.extract_name else None,
            'shares': predict_shares(doc) if self.extract_shares else None
        }

You can instantiate @weave.type() objects like this.

In [ ]:
regex_model = RegexModel(extract_name=False, extract_shares=True)

And then call methods on them like normal.

In [ ]:
regex_model.predict(example_doc)

If we change the model's configuration, or its definition (by changing its code), Weave will ensure that a new
version is published. Here we create a new model configuration and use it. There will be two versions of our RegexModel in the UI after this.

In [ ]:
regex_model = RegexModel(extract_name=True, extract_shares=True)
regex_model.predict(example_doc)

## Evaluation

Evaluation is used to give us apple-to-apples comparison of models.

You can think of evaluation as simply a function that takes a dataset and a model as input, and produces metrics as output.

We've defined an evaluation op that computes f1 scores and other metrics for text extraction problems, in the same directory as this notebook.

Just call it to run and track the evaluation for our model!

In [ ]:
from evaluate import evaluate_multi_task_f1
evaluate_multi_task_f1(dataset_ref, regex_model)

Click the link for the evaluation op printed above to see the results in the UI

## Using an LLM

Here we define a new model that uses OpenAI to extract the fields we want.

We use the `from weave.monitoring import openai` openai API wrapper to ensure the actual OpenAI calls are logged, in addition to the calls to our outer predict method.

We parameterize our model with the OpenAI model name, and the prompt template to use. So we'll get a new version of our model in the UI if we try with a different OpenAI model, or a different prompt template.

In [ ]:
@weave.type()
class OpenAIChatModel(weave.Model):
    model_name: str
    prompt_template: str

    @weave.op()
    def predict(self, doc: str) -> typing.Any:
        import json
        from weave.monitoring import openai
        response = openai.ChatCompletion.create(
            model=self.model_name,
            messages=[
                {'role': 'user',
                 'content': self.prompt_template.format(doc=doc)}])
        result = response['choices'][0]['message']['content']
        parsed = json.loads(result)
        return {
            'name': parsed['name'],
            'shares': int(parsed['shares'])
        }

In [ ]:
model = OpenAIChatModel(
    'gpt-3.5-turbo',
    "Extract company name (name) and number of shares (shares) from the following Articles of Incorporation document, as a json object: {doc}")

Let's call the model.

Sometimes this model results in an exception, because OpenAI returns invalid json, which we then try to json.loads. If you run the following cell a few times, you'll see this happen.

That's ok! Weave will track raised Exceptions from ops.

In [ ]:
try:
    model.predict(example_doc)
except Exception as e:
    print('Exception:', e)

Now let's evaluate the new model. 

In [ ]:
evaluate_multi_task_f1(dataset_ref, model)

The LLM model is much more accurate already, and there's plenty we can do from here to improve it.

Take a look at the UI. Click the row in the evaluation that was just created to see the details of this run. You'll see a trace of it's execution, which shows that some of the calls failed due to invalid json output.

Try to fix the prompt to improve the json output and try again!